In [21]:
import fiftyone as fo
from fiftyone import ViewField as F
import fiftyone.utils.data as foud
from object_detection import annotation_biigle as ab
from object_detection import fifty_one_utils as fou
from object_detection.cross_validation import k_fold_cross_validation
from object_detection.weightedDataset import YOLOWeightedDataset
from object_detection.spliter import dataset_tiler
import ultralytics.data.build as build
from ultralytics import YOLO
from mongoengine import connect
import os


# Convert Biigle csv annotation dataset to a yolov5 training dataset

In this section we will convert the Biigle csv annotation dataset into a yolo25 dataset usable in yolo models for object detections

In [2]:
# Get import image directory path
image_dir = "D:/KANADEEP/01_Donnee/02_Image/PL01/Annotations/Annotated_Images"

# Get import BIIGLE csv annotation report file path
report_file = "U:/10_THESE/Felix Navarro/01_Donnee/01_Plongee/PL01/04_Annotations/91_csv_image_annotation_report/78-catami-classification-scheme.csv"

# Get export directory for Yolov5 format dataset
export_dir = "D:/KANADEEP/01_Donnee/02_Image/PL01/Annotations/Kfold_Training"

# Get name for project
name = "pl01"

# Get column to extract label from
label_col = "label_hierarchy"

# Get label depth to extract label
level = 1

In [3]:
# Import unique classes from BIIGLE csv annotation report file according to asked or lowest annotation level depth
classes = ab.classes_BIIGLE(report_dir = report_file, label_col = label_col, level = level)


# Create Fiftyone dataset

In [4]:
# Connect to MongoDB
connect(
    db="fiftyone",  # Specify the name of your database (default for FiftyOne is 'fiftyone')
    host="mongodb://localhost:27017",  # MongoDB URI (adjust if using remote or custom ports)
)

# Configure FiftyOne to use a specific MongoDB URI
fo.config.database_uri = "mongodb://localhost:27017"


In [5]:
# Create fiftyone dataset
dataset = fo.Dataset()

# Get annotated images from image dir
annotated_images = ab.get_annotated_images(report_file, image_dir)
annotated_images_parser = foud.ImageSampleParser()

# Add annotated images to dataset
dataset.add_images(annotated_images, sample_parser = annotated_images_parser)

# Import BIIGLE csv annotation in fiftyone according to asked or lowest annotation level depth
annotations = fou.import_image_csv_report(image_dir = image_dir, report_file = report_file, level = level)

# Add annotation to image dataset
dataset.add_samples(annotations)

 100% |█████████████████| 608/608 [289.6ms elapsed, 0s remaining, 2.1K samples/s]     


100%|██████████| 608/608 [00:09<00:00, 60.91it/s]

   0% ||----------------|   1/608 [30.2ms elapsed, 18.3s remaining, 33.2 samples/s] 

 100% |█████████████████| 608/608 [9.5s elapsed, 0s remaining, 59.7 samples/s]      


['67bd8f1abac42f4a3c8b6f39',
 '67bd8f1abac42f4a3c8b6f3a',
 '67bd8f1abac42f4a3c8b6f3b',
 '67bd8f1abac42f4a3c8b6f3c',
 '67bd8f1abac42f4a3c8b6f3d',
 '67bd8f1abac42f4a3c8b6f3e',
 '67bd8f1abac42f4a3c8b6f3f',
 '67bd8f1abac42f4a3c8b6f40',
 '67bd8f1abac42f4a3c8b6f41',
 '67bd8f1abac42f4a3c8b6f42',
 '67bd8f1abac42f4a3c8b6f43',
 '67bd8f1abac42f4a3c8b6f44',
 '67bd8f1abac42f4a3c8b6f45',
 '67bd8f1abac42f4a3c8b6f46',
 '67bd8f1abac42f4a3c8b6f47',
 '67bd8f1abac42f4a3c8b6f48',
 '67bd8f1abac42f4a3c8b6f49',
 '67bd8f1abac42f4a3c8b6f4a',
 '67bd8f1abac42f4a3c8b6f4b',
 '67bd8f1abac42f4a3c8b6f4c',
 '67bd8f1abac42f4a3c8b6f4d',
 '67bd8f1abac42f4a3c8b6f4e',
 '67bd8f1abac42f4a3c8b6f4f',
 '67bd8f1abac42f4a3c8b6f50',
 '67bd8f1abac42f4a3c8b6f51',
 '67bd8f1abac42f4a3c8b6f52',
 '67bd8f1abac42f4a3c8b6f53',
 '67bd8f1abac42f4a3c8b6f54',
 '67bd8f1abac42f4a3c8b6f55',
 '67bd8f1abac42f4a3c8b6f56',
 '67bd8f1abac42f4a3c8b6f57',
 '67bd8f1abac42f4a3c8b6f58',
 '67bd8f1abac42f4a3c8b6f59',
 '67bd8f1abac42f4a3c8b6f5a',
 '67bd8f1abac4

In [6]:

dataset.default_classes = fou.get_classes(dataset)


['ECH_CRI_Feather stars',
 'WORMS_POL_Polychaetes',
 'CRU_PRA_Prawns/Shrimps/Mystids',
 'CNI_HYD_Hydrocorals',
 'CRU_CRA_Crabs',
 'MOL_GAS_Gastropods',
 'CNI_Cnidaria',
 'SP_CRU_Crust-like',
 'SP_CUP_Cup-like',
 'ECH_URC_Sea Urchins',
 'SP_ER_Erect',
 'FIS_Fishes',
 'SP_Sponges',
 'CNI_TUBA_Tube Anemones',
 'UNIDENT_Unidentified',
 'SP_MA_Massive',
 'CRU_Crustacea',
 'ECH_STAR_Sea Stars',
 'CNI_COL_Colonial Anemones',
 'ECH_OPH_Ophiuroids',
 'CNI_CO_Corals',
 'CNI_TRUA_True Anemones',
 'MOL_Molluscs',
 'ECH_SC_Sea Cucumbers']

In [7]:
dataset = dataset.match(F("detections.detections").length() != 0)

# Train Yolov11 model

In [8]:
# Build weighted dataset to compensate classes disparity
build.YOLODataset = YOLOWeightedDataset

In [9]:
# Get temporay directory
temp_dir = "D:/KANADEEP/01_Donnee/02_Image/PL01/Annotations/temp"

In [10]:
# Tile dataset to augment precision
tiled_dataset = dataset_tiler(dataset, temp_dir, 2000)

100%|██████████| 608/608 [01:44<00:00,  5.80it/s]

   0% ||--------------|    1/1203 [38.8ms elapsed, 46.7s remaining, 25.8 samples/s] 

 100% |███████████████| 1203/1203 [11.7s elapsed, 0s remaining, 74.7 samples/s]       


In [11]:
# Divide dataset for Kfold Cross Validation
ds_yamls = k_fold_cross_validation(test, export_dir)

C:\Users\fnavarro\ownCloud - Felix.Navarro@ifremer.fr@cloud.ifremer.fr\Script\YOLO-Detection-toolbox\object_detection\cross_validation.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  labels_df = labels_df.fillna(0.0)


ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
SP_CRU_Crust-like
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
SP_ER_Erect
ECH_CRI_Feather stars
SP_Sponges
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_CRI_Feather stars
SP_MA_Massive
UNIDENT_Unidentified
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OPH_Ophiuroids
ECH_OP

C:\Users\fnavarro\ownCloud - Felix.Navarro@ifremer.fr@cloud.ifremer.fr\Script\YOLO-Detection-toolbox\object_detection\cross_validation.py:40: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  folds_df[f"split_{i}"].loc[labels_df.iloc[train].inde

Copying images and exporting labels to new directories (YoloV5)


100%|██████████| 1203/1203 [14:26<00:00,  1.39it/s]


In [28]:
# Get project name
project_name = "KANADEEP"

# Get model path
dir = os.getcwd()
untrained_model_path = os.path.join(dir, 'model','yolov11l.pt')
untrained_model_path = "C:/Users/fnavarro/'ownCloud - Felix.Navarro@ifremer.fr@cloud.ifremer.fr'/Script/YOLO-Detection-toolbox/model/yolo11l.pt"

# Define your additional arguments here
batch = 16
epochs = 100
results = {}

In [29]:
for k in range(5):
    dataset_yaml = ds_yamls[k]
    model = YOLO(untrained_model_path, task="detect")
    model.train(data = dataset_yaml, epochs = epochs, batch = batch, project = project_name)  # include any train arguments
    results[k] = model.metrics  # save output metrics for further analysis

Ultralytics 8.3.78  Python-3.11.11 torch-2.5.1 CPU (Intel Core(TM) i5-10400H 2.60GHz)
engine\trainer: task=detect, mode=train, model=C:/Users/fnavarro/'ownCloud - Felix.Navarro@ifremer.fr@cloud.ifremer.fr'/Script/YOLO-Detection-toolbox/model/yolo11l.pt, data=D:\KANADEEP\01_Donnee\02_Image\PL01\Annotations\Kfold_Training\2025-02-25_5-Fold_Cross-val\split_1\split_1_dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=KANADEEP, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=Fa

100%|██████████| 755k/755k [00:00<00:00, 9.79MB/s]

Overriding model.yaml nc=80 with nc=23

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  2    173824  ultralytics.nn.modules.block.C3k2            [128, 256, 2, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  2    691712  ultralytics.nn.modules.block.C3k2            [256, 512, 2, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              


  6                  -1  2   2234368  ultralytics.nn.modules.block.C3k2            [512, 512, 2, True]           
  7                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  8                  -1  2   2234368  ultralytics.nn.modules.block.C3k2            [512, 512, 2, True]           
  9                  -1  1    656896  ultralytics.nn.modules.block.SPPF            [512, 512, 5]                 
 10                  -1  2   1455616  ultralytics.nn.modules.block.C2PSA           [512, 512, 2]                 
 11                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 12             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 13                  -1  2   2496512  ultralytics.nn.modules.block.C3k2            [1024, 512, 2, True]          
 14                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None

train: Scanning D:\KANADEEP\01_Donnee\02_Image\PL01\Annotations\Kfold_Training\2025-02-25_5-Fold_Cross-val\split_1\train\labels... 962 images, 0 backgrounds, 0 corrupt: 100%|██████████| 962/962 [00:01<00:00, 757.48it/s]


train: New cache created: D:\KANADEEP\01_Donnee\02_Image\PL01\Annotations\Kfold_Training\2025-02-25_5-Fold_Cross-val\split_1\train\labels.cache


val: Scanning D:\KANADEEP\01_Donnee\02_Image\PL01\Annotations\Kfold_Training\2025-02-25_5-Fold_Cross-val\split_1\val\labels... 241 images, 0 backgrounds, 0 corrupt: 100%|██████████| 241/241 [00:00<00:00, 948.82it/s]


val: New cache created: D:\KANADEEP\01_Donnee\02_Image\PL01\Annotations\Kfold_Training\2025-02-25_5-Fold_Cross-val\split_1\val\labels.cache
Plotting labels to KANADEEP\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00037, momentum=0.9) with parameter groups 167 weight(decay=0.0), 174 weight(decay=0.0005), 173 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to KANADEEP\train
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/61 [01:31<?, ?it/s]


KeyboardInterrupt: 